In [1]:
from langchain_groq import ChatGroq
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from IPython.display import Image, display

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [ ]:
groq_api_key = 'gsk.....PTO'
llm = ChatGroq(groq_api_key=groq_api_key, model_name="Gemma2-9b-It")
langsmith='lsv......26d'

In [3]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    sentiment: str

# 1. Preprocessing Node
def preprocess(state: State) -> State:
    cleaned = state["messages"][-1].content.strip()
    state["messages"][-1].content = cleaned
    return state

# 2. Sentiment Analysis Node
def analyze_sentiment(state: State) -> State:
    msg = state["messages"][-1].content
    state["sentiment"] = "positive" if "good" in msg else "neutral"
    return state

# 3. Chatbot Node
def chatbot(state: State) -> State:
    return {"messages": llm.invoke(state['messages'])}

# 4. Logging Node
def logger(state: State) -> State:
    print(f"LOG: {state['messages'][-1].content}, Sentiment: {state.get('sentiment')}")
    return state

In [4]:
# Build the graph
builder = StateGraph(State)
builder.add_node("preprocess", preprocess)
builder.add_node("analyze_sentiment", analyze_sentiment)  # renamed
builder.add_node("chatbot", chatbot)
builder.add_node("logger", logger)

In [5]:
# Define flow
builder.add_edge(START, "preprocess")
builder.add_edge("preprocess", "analyze_sentiment")  # renamed
builder.add_edge("analyze_sentiment", "chatbot")     # renamed
builder.add_edge("chatbot", "logger")
builder.add_edge("logger", END)

In [6]:
graph = builder.compile()

In [9]:
# Draw the graph
try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception :
    pass

In [8]:
# Invoke the graph
input_message = "hi i am Abhinay."
final_state = graph.invoke({"messages": ("user", input_message)})

# Print the final state
print("Final State:", final_state)
print("Chatbot's response:", final_state['messages'][-1].content)
print("Detected Sentiment:", final_state['sentiment'])

input_message_2 = "This is a neutral message."
final_state_2 = graph.invoke({"messages": ("user", input_message_2)})
print("Final State 2:", final_state_2)
print("Chatbot's response 2:", final_state_2['messages'][-1].content)
print("Detected Sentiment 2:", final_state_2['sentiment'])

LOG: Hello Abhinay! It's nice to meet you. What can I do for you today?
, Sentiment: neutral
Final State: {'messages': [HumanMessage(content='hi i am Abhinay.', additional_kwargs={}, response_metadata={}, id='a82f118e-be44-4b89-a9a6-395ea0479089'), AIMessage(content="Hello Abhinay! It's nice to meet you. What can I do for you today?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 16, 'total_tokens': 40, 'completion_time': 0.043636364, 'prompt_time': 0.001253877, 'queue_time': 0.250822503, 'total_time': 0.044890241}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--6db33f25-5dfe-4016-8d6a-f4d75daf2d09-0', usage_metadata={'input_tokens': 16, 'output_tokens': 24, 'total_tokens': 40})], 'sentiment': 'neutral'}
Chatbot's response: Hello Abhinay! It's nice to meet you. What can I do for you today?

Detected Sentiment: neutral
LOG: I understand. 

Is there anything e